In [ ]:
import os
os.environ['HF_HOME'] = '/home/linux_user/EAGLE/eagle/model'

base_model_paths = ['yuhuili/EAGLE-Vicuna-7B-v1.3',
                    'yuhuili/EAGLE-Vicuna-13B-v1.3',
                    'yuhuili/EAGLE-Vicuna-33B-v1.3',
                    'yuhuili/EAGLE3-DeepSeek-R1-Distill-LLaMA-8B']

EAGLE_model_paths = ['lmsys/vicuna-7b-v1.3',
                     'lmsys/vicuna-13b-v1.3',
                     'lmsys/vicuna-33b-v1.3',
                     'deepseek-ai/DeepSeek-R1-Distill-Llama-8B']

# Code From https://github.com/SafeAILab/EAGLE
from eagle.model.ea_model import EaModel
from fastchat.model import get_conversation_template
model = EaModel.from_pretrained(
    base_model_path=base_model_paths[0],
    ea_model_path=EAGLE_model_paths[0],
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    device_map="auto",
    total_token=-1,
    offload_folder="offload" # https://github.com/nomic-ai/gpt4all/issues/239
)

model.eval()
your_message="Hello"
conv = get_conversation_template("vicuna")
conv.append_message(conv.roles[0], your_message)
conv.append_message(conv.roles[1], None)
prompt = conv.get_prompt()
input_ids=model.tokenizer([prompt]).input_ids
input_ids = torch.as_tensor(input_ids).cuda()
output_ids=model.eagenerate(input_ids,temperature=0.5,max_new_tokens=512)
output=model.tokenizer.decode(output_ids[0])

In [ ]:
'''
Note: May Need To Add Something like the following to cnets.py:

if "deepseek-ai" in path:
    path = os.path.join(os.path.dirname(__file__), "hub/models--deepseek-ai--DeepSeek-R1-Distill-Llama-8B/snapshots/6a6f4aa4197940add57724a7707d069478df56b1/")
elif "EAGLE3-DeepSeek" in path:
    path = os.path.join(os.path.dirname(__file__), "hub/models--yuhuili--EAGLE3-DeepSeek-R1-Distill-LLaMa-8B/snapshots/a55854d50efdf4fd8b39fc303a365bca8b4ff270/")

'''